<a href="https://colab.research.google.com/github/majidsadrii/persian_vehicle_license/blob/main/training_custome_yolov7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1.parameters and imports


In [ ]:
clone_model  = False
project_dir =  '/content/drive/MyDrive/car_plate' 

In [ ]:
import sys
import shutil
import xml.etree.ElementTree as ET
!pip install colorama
from colorama import Fore
from tqdm import tqdm
import random
import os 
import torch
print(f"Python version: {sys.version}, {sys.version_info} ")
print(f"Pytorch version: {torch.__version__} ")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Python version: 3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0], sys.version_info(major=3, minor=8, micro=10, releaselevel='final', serial=0) 
Pytorch version: 1.13.1+cu116 


In [ ]:
!nvidia-smi

Tue Mar  7 07:43:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
yolo_path = os.path.join(project_dir, "yolov7")
if clone_model :
 
  os.chdir(project_dir)
  !git clone https://github.com/WongKinYiu/yolov7
  
  if  not os.path.isfile( os.path.join(yolo_path, "yolov7.pt")):
    os.chdir(yolo_path)
    !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [ ]:
os.chdir(yolo_path)
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2.Dataset

## 2.1.download datasets from kaggle

In [ ]:
if  not os.path.isfile( '/content/drive/MyDrive/car_plate/data/car-plate-detection.zip'):
  ! pip install -q kaggle
  from google.colab import files
  files.upload()
  ! mkdir ~/.kaggle
  ! cp kaggle.json ~/.kaggle/
  ! chmod 600 ~/.kaggle/kaggle.json
  ! kaggle datasets list
  !kaggle datasets download -d andrewmvd/car-plate-detection
  !kaggle datasets download -d samyarr/iranvehicleplatedataset
  !cp -r '/content/car-plate-detection.zip' '/content/drive/MyDrive/car_plate/data'
  !cp -r '/content/iranvehicleplatedataset.zip' '/content/drive/MyDrive/car_plate/data'
else:
  print(Fore.CYAN,'datasets already exist')

 datasets already exist


## 2.2.prepraing dataset

In [ ]:
if  not os.path.isfile('/content/dataset/Vehicle Plates/Vehicle Plates/192.png'):
  os.makedirs('/content/dataset')
  ! unzip -q '/content/drive/MyDrive/car_plate/data/car-plate-detection.zip' -d '/content/dataset'
  ! unzip -q '/content/drive/MyDrive/car_plate/data/iranvehicleplatedataset.zip' -d '/content/dataset'
else:
  print(Fore.CYAN,'datasets are already extracted')

 datasets are already extracted


In [ ]:
'''
reading all files
'''
files_1 = os.listdir('/content/dataset/Vehicle Plates/Vehicle Plates')
files_1  = [os.path.join('/content/dataset/Vehicle Plates/Vehicle Plates', i) for i in files_1 ]
annotation_1 = os.listdir('/content/dataset/Vehicle Plates annotations/Vehicle Plates annotations')
annotation_1  = [os.path.join('/content/dataset/Vehicle Plates annotations/Vehicle Plates annotations', i) for i in annotation_1 ]
files_2 = os.listdir('/content/dataset/images')
files_2  = [os.path.join('/content/dataset/images', i) for i in files_2 ]
annotation_2 = os.listdir('/content/dataset/annotations')
annotation_2  = [os.path.join('/content/dataset/annotations', i) for i in annotation_2 ]

In [ ]:
for el in files_2:
  files_1.append(el)
for el in annotation_2:
  annotation_1.append(el)
annotation_1.sort()  
files_1.sort()
print('num ann = {} num of images = {}'.format(len(annotation_1),len(files_1)))
assert len(files_1) ==len(annotation_1),  'there must be the same number of images and annotations'
print(annotation_1[-160])
print(files_1[-160])


num ann = 746 num of images = 746
/content/dataset/annotations/Cars344.xml
/content/dataset/images/Cars344.png


In [ ]:
temp = list(zip(files_1,annotation_1))
random.shuffle(temp)
res1, res2 = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
files_1, annotation_1 = list(res1), list(res2)
print(annotation_1[-160])
print(files_1[-160])

/content/dataset/annotations/Cars361.xml
/content/dataset/images/Cars361.png


## 2.3. writing annotation in yolov7 format

In [ ]:
def create_annotations(image_path,image_annotation,dataset_path,data_type):
  print('start to create {} set'.format(data_type))
  if  not os.path.exists(dataset_path):
    os.mkdir(dataset_path)
  tem_path = os.path.join(dataset_path,data_type) 
  labels_path = os.path.join(tem_path,'labels')
  img_path = os.path.join(tem_path,'images')
  if   os.path.exists(tem_path ):
    pass

  else:
        os.mkdir(tem_path)       
        os.mkdir(img_path )        
        os.mkdir(labels_path)
  for index in tqdm(range(len(image_path))):
     
      current_img = image_path[index].split('.')[0].split('/')[-1]
      current_annot = image_annotation[index].split('.')[0].split('/')[-1]

      if current_annot != current_img:
        print(Fore.RED,'warning!!! file name is {} while annot name is {}'.format(current_img,current_annot))
      des = os.path.join(img_path,current_img+'.png' )
     
      shutil.copyfile(image_path[index],des  ) 
          
      root = ET.parse(image_annotation[index]).getroot()
      
      objects = root.findall('object')
      sz = root.find('size')
      h = sz.find('height').text.strip()
      w = sz.find('width').text.strip()
      w = int(w)
      h = int(h)
      annotation = ''
      for obj in objects:         
          bbox = obj.find('bndbox')
          class_id = 0
          xmin = bbox.find('xmin').text.strip()
          ymin = bbox.find('ymin').text.strip()
          xmax = bbox.find('xmax').text.strip()
          ymax = bbox.find('ymax').text.strip()

          X_m = str(int((int(xmin)+int(xmax))/2)/w)
          Y_m = str(int((int(ymin)+int(ymax))/2)/h)
          obj_w= str((int(xmax) -int(xmin))/(w))
          obj_h= str((int(ymax) -  int(ymin))/(h))
         
          annotation += '{} {} {} {} {}'.format(class_id, X_m , Y_m,obj_w,obj_h)

      anno_path =    os.path.join(labels_path,current_annot+'.txt')
     
      with open(anno_path, 'a') as f:
         f.write(annotation)
  t1 = os.listdir(labels_path)
  t1 = [i.split('.')[0] for i in t1]
  t2 = os.listdir(img_path)
  t2 = [i.split('.')[0] for i in t2]
  
  if len(t1) !=len(t2):
      print(Fore.RED,'sth is wrong',Fore.BLACK)
  t1.sort()
  t2.sort()

  

  for item in t1:
    if item not in t2:
      print(' item {} not not exist in txt'.format(item))
  for item in t2:
    if item not in t1:
      print(' item {} not not exist in png'.format(item))


    

try:
  shutil.rmtree('/content/custome_dataset')
except:
  pass
train_c = int(0.8*len(files_1)) 
val_c = int(0.9*len(files_1))   
create_annotations(files_1[:train_c] ,      annotation_1[:train_c] ,dataset_path='/content/custome_dataset',data_type='train')
create_annotations(files_1[train_c:val_c],annotation_1[train_c:val_c],dataset_path='/content/custome_dataset',data_type='val')
create_annotations(files_1[val_c:],         annotation_1[val_c:],dataset_path='/content/custome_dataset',data_type='test')




start to create train set


100%|██████████| 596/596 [00:02<00:00, 241.78it/s]


start to create val set


100%|██████████| 75/75 [00:00<00:00, 394.07it/s]


start to create test set


100%|██████████| 75/75 [00:00<00:00, 332.13it/s]


In [ ]:
!pip install pyyaml
import yaml

dict_file = {'names' : ['car_plate'],
'nc' : int(1),
'test': '/content/custome_dataset/test/images',
'train':'/content/custome_dataset/train/images',
'val':'/content/custome_dataset/val/images'}
yml_loc = '/content/custome_dataset' +'/data.yml'
with open(yml_loc, 'w') as file:
    documents = yaml.dump(dict_file, file)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 3.train the model

In [ ]:
os.chdir(yolo_path)
!python train.py --batch 16 \
--cfg cfg/training/yolov7.yaml \
--epochs 15 \
--data /content/custome_dataset/data.yml\
--weights 'yolov7.pt' --device 0 
     


YOLOR 🚀 v0.1-122-g3b41c2c torch 1.13.1+cu116 CUDA:0 (Tesla T4, 15101.8125MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='cfg/training/yolov7.yaml', data='/content/custome_dataset/data.yml', device='0', entity=None, epochs=15, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp6', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, v5_metric=False, weights='yolov7.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2023-03-07 07:44:00.725505: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary 